In [128]:
import pandas as pd
from sqlalchemy import create_engine
from wordcloud import WordCloud
import re

In [ ]:
folder = "XXX/"
file='XXX.csv'

In [ ]:
df=pd.read_csv(folder+file)
df

In [ ]:
#get the date right and two columns for date and year
df.publication_date = pd.to_datetime(df['publication_date'])

df['just_date'] = df['publication_date'].dt.date
df['just_year']= df['publication_date'].dt.year

Add useful filters (remember to filter by language!- langauge!)

In [ ]:
#filter by date
dfmodern=df.loc[df.publication_date > "30/04/2021"]
len(dfmodern)

In [148]:
#filter by language
dfmodernEN=df.loc[df.langauge == "en"]

In [ ]:
#filter by country
dfNGA=dfmodern.loc[dfmodern.country_iso3 == "NGA"]
dfNGA

Trying Topic modeling 

In [ ]:
#from: https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0
# Remove the columns
papers = dfmodernEN.drop(columns=['publication_date', 'country_iso3','is_homepage','crawl_date','langauge','just_year'], axis=1)
#Check this next one is only for a sample.
#papers = dfNGA.drop(columns=['publication_date', 'country_iso3','is_homepage','crawl_date','langauge','just_year'], axis=1).sample(200)
# Print out the first rows of papers
len(papers)
#papers.head()

In [ ]:
# Load the regular expression library

# Remove punctuation
papers['paper_text_processed'] = \
papers['news_content'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
papers['paper_text_processed'] = \
papers['paper_text_processed'].map(lambda x: x.lower())
# Print out the first rows of papers
papers['paper_text_processed'].head()

In [ ]:
#pip install wordcloud

In [ ]:
# Import the wordcloud library
from wordcloud import WordCloud


# Join the different processed titles together.
long_string = ','.join(list(papers['paper_text_processed'].values))
# Create a WordCloud object
wordcloud = WordCloud(background_color="white", max_words=5000, contour_width=3, contour_color='steelblue')
# Generate a word cloud
wordcloud.generate(long_string)
# Visualize the word cloud
wordcloud.to_image()

In [ ]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'said'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = papers.paper_text_processed.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

In [ ]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

In [ ]:
from pprint import pprint
from pprint import pprint


# number of topics
num_topics = 7


# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)


# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
#import gensim.pyLDAvis
import pyLDAvis.gensim_models
#from pyLDAvis import gensim
import pickle 
import pyLDAvis


# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join(folder+str(num_topics)+"Allnews")
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, folder+ str(num_topics)+"Allnews" +'.html')
LDAvis_prepared

Interpreting: give each topic a title (interpret the clusters)

1- Economy - Politics

2- Society and social media

3- Kaduna kills in Nigeria

4- Sudan Constitutional Period

5- Champions League

6- Unrest Gauteng SA.

7- Covid

In [ ]:

lda_model[corpus][58319]

In [291]:
#check this: 
    #https://towardsdatascience.com/topic-modelling-in-python-with-spacy-and-gensim-dc8f7748bdbf

Bring topic back to df

In [ ]:
dfmodernEN['topic'] = [sorted(lda_model[corpus][text])[0][0] for text in range(len(dfmodernEN['news_content']))]
dfmodernEN

In [ ]:
#count data by topic frequency
dfmodernEN.topic.value_counts()

In [324]:
#save the new dataframe
dfmodernEN.to_csv(folder+str(num_topics)+"XXXXX.csv", encoding='utf-8')